In [2]:
import os
#my_dir = "/home/oddc/crypto_robot_live/"
my_dir = "/Users/olivierdedecker/Documents/00_Dev/Python/Crypto_Robot_live/"
lib_file_path = os.path.join(my_dir, 'backtest_tools', 'utilities')
import sys
sys.path.append(lib_file_path)
#from get_data import get_historical_from_db
from backtesting import basic_single_asset_backtest, plot_wallet_vs_asset, get_metrics, get_n_columns, plot_sharpe_evolution, plot_bar_by_month
from data_manager import ExchangeDataManager
from perp_bybit import *
import pandas as pd
import ta
#import ccxt
import json
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# --- PARAMETERS & VARIABLES ---
# -- Account --
secret_file_path = os.path.join(my_dir, 'live_tools', 'secret.json')
f = open(secret_file_path)
secret = json.load(f)
f.close()

exchange_name = 'bybit'
account_to_select = 'testnet_account'
production = True# connect exchange
bybit = PerpBybit(
    apiKey=secret[account_to_select]["apiKey"],
    secret=secret[account_to_select]["secret"],
    default_type='swap',
    is_real=secret[account_to_select]["is_real"]
)

hey .. set sandbox mode ON


In [4]:
class envelope_strat():
    def __init__(
        self,
        df,
        ema_shifts = [0.05, 0.1, 0.15],
        ema_period = 5,
    ):
        self.df = df
        self.ema_shifts = ema_shifts
        self.ema_period = ema_period
        self.sell_ema_values = {}
        self.buy_ema_values = {}
        self.nLevel = len(ema_shifts)
        self.position_type = ["long", "short"]
        self.open_position_asap = True
        self.close_position_with_indicator = False
        
    def populate_indicators(self, show_log=False):
        # -- Clear dataset --
        df = self.df
        df.drop(columns=df.columns.difference(['open','high','low','close','volume']), inplace=True)
        
        # -- Populate indicators --
        df['ema_base'] = ta.trend.ema_indicator(close=df['close'], window=self.ema_period)

        for i, shift in enumerate(self.ema_shifts, start=1):
            df[f'ema_high_{i}'] = df['ema_base'] * (1 + shift)
            df[f'ema_low_{i}'] = df['ema_base'] * (1 - shift)
            self.sell_ema_values[f'ema_high_{i}'] = bybit.convert_price_to_precision(pair, df.iloc[-1][f'ema_high_{i}'])
            self.buy_ema_values[f'ema_low_{i}'] = bybit.convert_price_to_precision(pair, df.iloc[-1][f'ema_low_{i}'])
        
        #df = get_n_columns(df, ["super_trend_direction", "ema_short", "ema_long"], 1)
        
        # -- Log --
        if(show_log):
            print(df)
        
        self.df = df    
        return self.df
    
    def populate_buy_sell(self, show_log=False): 
        df = self.df
        # -- Initiate populate --
        for i in range(1, self.nLevel+1):
            df[f"open_long_limit_{i}"] = False
            df[f"open_short_limit_{i}"] = False
        df["close_long_limit"] = False
        df["close_short_limit"] = False
        
        # -- Populate open long and short limits --
        for i in range(1, self.nLevel+1):
            if 'long' in self.position_type and self.open_position_asap:
                df.loc[
                    (df[f'ema_low_{i}'] > df['low']) 
                    , f"open_long_limit_{i}"
                ] = True
            if 'short' in self.position_type and self.open_position_asap:
                df.loc[
                    (df[f'ema_high_{i}'] < df['high']) 
                    , f"open_short_limit_{i}"
                ] = True
        
        # -- Populate close long and short limits --
        if not self.close_position_with_indicator:
            df.loc[
                (df['ema_base'] < df['high'])
                , "close_long_limit"
            ] = True
            df.loc[
                (df['ema_base'] > df['low'])
                , "close_short_limit"
            ] = True
        
        # -- Log --
        if(show_log):
            for i in range(1, self.nLevel+1):
                print(f"Number of Open LONG {i} conditions :",len(df.loc[df[f"open_long_limit_{i}"]==True]))
                print(f"Number of Open SHORT {i} conditions :",len(df.loc[df[f"open_short_limit_{i}"]==True]))
            print(f"Number of Close LONG conditions :",len(df.loc[df[f"close_long_limit"]==True]))
            print(f"Number of Close SHORT conditions :",len(df.loc[df[f"close_short_limit"]==True]))

        
        self.df = df   
        return self.df
        
    def run_backtest(self, initial_wallet=1000, return_type="metrics"):
        dt = self.df[:]
        wallet = initial_wallet
        maker_fee = 0
        taker_fee = 0.0007
        trades = []
        days = []
        current_day = 0
        previous_day = 0
        current_position = None
        
        for index, row in dt.iterrows():
            
            # -- Add daily report --
            current_day = index.day
            if previous_day != current_day:
                temp_wallet = wallet
                if current_position:
                    if current_position['side'] == "LONG":
                        close_price = row['close']
                        trade_result = (close_price - current_position['price']) / current_position['price']
                        temp_wallet += temp_wallet * trade_result
                        fee = temp_wallet * taker_fee
                        temp_wallet -= fee
                    
                days.append({
                    "day":str(index.year)+"-"+str(index.month)+"-"+str(index.day),
                    "wallet":temp_wallet,
                    "price":row['close']
                })
            previous_day = current_day

            if current_position:
            # -- Check for closing position --
                if current_position['side'] == "LONG":                     

                    # -- Close LONG limit --
                    if row['close_long_limit']:
                        close_price = row['n1_ema_short']
                        trade_result = (close_price - current_position['price']) / current_position['price']
                        wallet += wallet * trade_result
                        fee = wallet * maker_fee
                        wallet -= fee
                        trades.append({
                            "open_date": current_position['date'],
                            "close_date": index,
                            "position": "LONG",
                            "open_reason": current_position['reason'],
                            "close_reason": "Limit",
                            "open_price": current_position['price'],
                            "close_price": close_price,
                            "open_fee": current_position['fee'],
                            "close_fee": fee,
                            "open_trade_size":current_position['size'],
                            "close_trade_size": wallet,
                            "wallet": wallet
                        })
                        current_position = None

            # -- Check for opening position --
            else:
                # Open LONG and SHORT limits
                for i in range(1, self.nLevel+1): 
                    if row[f'open_long_limit_{i}']:
                        open_price = row[f'ema_low_{i}']
                        fee = (wallet/self.nLevel) * maker_fee
                        wallet -= fee
                        pos_size = wallet
                        current_position = {
                            "size": pos_size,
                            "date": index,
                            "price": open_price,
                            "fee":fee,
                            "reason": "Limit",
                            "side": "LONG"
                        }
                    
                    
        df_days = pd.DataFrame(days)
        df_days['day'] = pd.to_datetime(df_days['day'])
        df_days = df_days.set_index(df_days['day'])

        df_trades = pd.DataFrame(trades)
        df_trades['open_date'] = pd.to_datetime(df_trades['open_date'])
        df_trades = df_trades.set_index(df_trades['open_date'])   
        
        if return_type == "metrics":
            return get_metrics(df_trades, df_days) | {
                "wallet": wallet,
                "trades": df_trades,
                "days": df_days
            }  
        else:
            return True   
        

In [5]:
# pair = "LTC/USDT"
# tf = "1h"

# df = get_historical_from_db(
#     ccxt.binance(), 
#     pair,
#     tf,
#     path="../../database/"
# )

exchange_name = "binance"
interval = "1h"
pair = 'BTC/USDT:USDT'
exchange_data_path = os.path.join(my_dir, 'my_data', 'database', 'exchanges')

exchange = ExchangeDataManager(exchange_name=exchange_name, path_download=exchange_data_path)
await exchange.download_data(coins=[pair], intervals=[interval])
df = exchange.load_data(coin=pair, interval=interval)

print("Data load 100%")

/Users/olivierdedecker/Documents/00_Dev/Python/Crypto_Robot_live/backtest_tools/utilities/data_manager.py:237: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file_name, index_col=0, parse_dates=True)


	Récupération pour la paire BTC/USDT:USDT en timeframe 1h sur l'exchange binance...


100%|██████████| 1/1 [00:01<00:00,  1.16s/it]

Data load 100%



/Users/olivierdedecker/Documents/00_Dev/Python/Crypto_Robot_live/backtest_tools/utilities/data_manager.py:114: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file_name, index_col=0, parse_dates=True)


In [6]:
strat = envelope_strat(
    df = df.loc[:],
    ema_shifts = [0.05, 0.1, 0.15],
    ema_period = 5,
)

strat.populate_indicators()
strat.populate_buy_sell(show_log=True)


Number of Open LONG 1 conditions : 114
Number of Open SHORT 1 conditions : 55
Number of Open LONG 2 conditions : 18
Number of Open SHORT 2 conditions : 3
Number of Open LONG 3 conditions : 7
Number of Open SHORT 3 conditions : 1
Number of Close LONG conditions : 30112
Number of Close SHORT conditions : 29727


,open,high,low,close,volume,ema_base,ema_high_1,ema_low_1,ema_high_2,ema_low_2,ema_high_3,ema_low_3,open_long_limit_1,open_short_limit_1,open_long_limit_2,open_short_limit_2,open_long_limit_3,open_short_limit_3,close_long_limit,close_short_limit
date,,,,,,,,,,,,,,,,,,,,
2019-09-08 17:00:00,10000.00,10000.00,10000.00,10000.00,0.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False
2019-09-08 18:00:00,10000.00,10000.00,10000.00,10000.00,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False
2019-09-08 19:00:00,10344.77,10357.53,10337.43,10340.12,471.659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False
2019-09-08 20:00:00,10340.12,10368.64,10334.54,10351.42,583.271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False
2019-09-08 21:00:00,10351.42,10391.90,10324.77,10391.90,689.759,10259.114815,10772.070556,9746.159074,11285.026296,9233.203333,11797.982037,8720.247593,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-05 03:00:00,30831.00,30875.20,30815.00,30857.30,3839.533,30817.695160,32358.579918,29276.810402,33899.464676,27735.925644,35440.349434,26195.040886,False,False,False,False,False,False,True,True
2023-07-05 04:00:00,30857.30,30862.60,30805.20,30825.10,3954.058,30820.163440,32361.171612,29279.155268,33902.179784,27738.147096,35443.187956,26197.138924,False,False,False,False,False,False,True,True
2023-07-05 05:00:00,30825.10,30858.90,30780.00,30804.50,4570.184,30814.942293,32355.689408,29274.195179,33896.436523,27733.448064,35437.183637,26192.700949,False,False,False,False,False,False,True,True


In [9]:
bt_result = strat.run_backtest(initial_wallet=1000, return_type="metrics")


KeyError: 'open_date'

In [ ]:
df_trades, df_days = basic_single_asset_backtest(trades=bt_result['trades'], days=bt_result['days'])
plot_wallet_vs_asset(df_days=df_days)

In [ ]:
plot_bar_by_month(df_days=df_days)